In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
imdb=keras.datasets.imdb
(train_data,train_labels), (test_data, test_labels)=imdb.load_data(num_words=10000)

print(f"Training entries {len(train_data)}. Labels: {len(train_labels)}")
print(len(train_data[0]), len(train_data[1]))

17465344/17464789 [==============================] - 6s 0us/step
Training entries 25000. Labels: 25000
218 189


In [3]:
word_index=imdb.get_word_index()

word_index= {k:(v+3) for k,v in word_index.items()}
word_index["<pad>"]=0
word_index["<Start>"]=1
word_index["<unk>"]=2
word_index["<unused>"]=3

1646592/1641221 [==============================] - 1s 0us/step


In [4]:
reverse_word_index=dict([(value,key) for (key, value) in word_index.items()])

def decode_reveiw(text):
  return" ".join([reverse_word_index.get(i,"?") for i in text])

In [6]:
print(train_data[0])
print(decode_reveiw(train_data[0]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
<Start> this film was just brilliant casting location scener

In [7]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,
                                                     value=word_index["<pad>"],
                                                     padding="post",
                                                     maxlen=256)


test_data=keras.preprocessing.sequence.pad_sequences(test_data,
                                                     value=word_index["<pad>"],
                                                     padding="post",
                                                     maxlen=256)
print(len(train_data[0]), len(train_data[1]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

In [8]:
###neural network
vocab_size=10000
#inputlayer
model=keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
#hiddenlayer
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
#output layer
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [9]:
#loss function
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["acc"])


x_val=train_data[:10000]
partial_x_train=train_data[10000:]


y_val=train_labels[:10000]
partial_y_train=train_labels[10000:]

In [10]:
history=model.fit(partial_x_train,
                  partial_y_train,
                  epochs=75,
                  batch_size=75,
                  validation_data=(x_val, y_val),
                  verbose=1)


Epoch 1/75
200/200 [==============================] - 1s 6ms/step - loss: 0.6660 - acc: 0.6889 - val_loss: 0.6055 - val_acc: 0.7887
Epoch 2/75
200/200 [==============================] - 1s 3ms/step - loss: 0.4880 - acc: 0.8336 - val_loss: 0.4062 - val_acc: 0.8526
Epoch 3/75
200/200 [==============================] - 1s 3ms/step - loss: 0.3318 - acc: 0.8818 - val_loss: 0.3300 - val_acc: 0.8706
Epoch 4/75
200/200 [==============================] - 1s 3ms/step - loss: 0.2618 - acc: 0.9020 - val_loss: 0.2967 - val_acc: 0.8814
Epoch 5/75
200/200 [==============================] - 1s 3ms/step - loss: 0.2193 - acc: 0.9200 - val_loss: 0.2839 - val_acc: 0.8861
Epoch 6/75
200/200 [==============================] - 1s 4ms/step - loss: 0.1887 - acc: 0.9329 - val_loss: 0.2798 - val_acc: 0.8874
Epoch 7/75
200/200 [==============================] - 1s 3ms/step - loss: 0.1640 - acc: 0.9450 - val_loss: 0.2826 - val_acc: 0.8884
Epoch 8/75
200/200 [==============================] - 1s 4ms/step - loss: 0.

200/200 [==============================] - 1s 4ms/step - loss: 7.5653e-05 - acc: 1.0000 - val_loss: 1.3500 - val_acc: 0.8526
Epoch 63/75
200/200 [==============================] - 1s 4ms/step - loss: 6.8942e-05 - acc: 1.0000 - val_loss: 1.3386 - val_acc: 0.8523
Epoch 64/75
200/200 [==============================] - 1s 4ms/step - loss: 5.2534e-05 - acc: 1.0000 - val_loss: 1.3490 - val_acc: 0.8528
Epoch 65/75
200/200 [==============================] - 1s 4ms/step - loss: 5.5577e-05 - acc: 1.0000 - val_loss: 1.3622 - val_acc: 0.8526
Epoch 66/75
200/200 [==============================] - 1s 4ms/step - loss: 4.7452e-05 - acc: 1.0000 - val_loss: 1.3845 - val_acc: 0.8531
Epoch 67/75
200/200 [==============================] - 1s 4ms/step - loss: 3.5693e-05 - acc: 1.0000 - val_loss: 1.3887 - val_acc: 0.8522
Epoch 68/75
200/200 [==============================] - 1s 5ms/step - loss: 3.6442e-05 - acc: 1.0000 - val_loss: 1.4267 - val_acc: 0.8528
Epoch 69/75
200/200 [==============================] 

In [11]:
results=model.evaluate(test_data,test_labels)
print(results)

782/782 [==============================] - 1s 817us/step - loss: 1.6194 - acc: 0.8387
[1.6194162368774414, 0.8387200236320496]
